In [3]:
# Extract final score
import pandas as pd
import os

auto_data = pd.read_csv(r"F:\Projects\LLMValueInvestigation\code\human_evaluation\outputs\Mixtral-8x7B-Instruct-v0.1-bnb-4bit\Chinese\mistral\qa_score.csv", index_col=0)

print(auto_data)

def findAllFile(base):
    for root, ds, fs in os.walk(base):
        for f in fs:
            if f.endswith("scores.csv"):
                fullname = os.path.join(root, f)
                yield fullname
            
base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\human_eval\\Mixtral-8x7B-Instruct-v0.1-bnb-4bit\\Chinese\\mistral\\"
human_data = pd.read_csv(base + "extra_score.csv", index_col=0).fillna(0)
print(human_data)
auto_data += human_data

print(auto_data)

auto_data.to_csv(base+"final_score.csv", index=True)

     0   1   2   3   4  5  6  7  8  9  10  11  12
0    0   0   0   0   0  0  0  0  0  0   0   0   0
1    0  20   0   0   0  0  0  0  0  0   0   0   0
2    0   1  19   0   0  0  0  0  0  0   0   0   0
3    0   1  19   0   0  0  0  0  0  0   0   0   0
4    0   0   3  17   0  0  0  0  0  0   0   0   0
..  ..  ..  ..  ..  .. .. .. .. .. ..  ..  ..  ..
536  0   0  19   0   0  0  0  0  0  0   0   0   0
537  0   0   3   1  14  1  0  0  0  0   0   0   0
538  0   0   4  15   0  0  0  0  0  0   0   0   0
539  3  16   0   0   0  0  0  0  0  0   0   0   0
540  0   0   2   4  13  0  0  0  0  0   0   0   0

[541 rows x 13 columns]
     0  1  2  3  4  5  6  7  8  9  10  11  12
0    0  0  0  0  0  0  0  0  0  0   0   0   0
1    0  0  0  0  0  0  0  0  0  0   0   0   0
2    0  0  0  0  0  0  0  0  0  0   0   0   0
3    0  0  0  0  0  0  0  0  0  0   0   0   0
4    0  0  0  0  0  0  0  0  0  0   0   0   0
..  .. .. .. .. .. .. .. .. .. ..  ..  ..  ..
536  0  0  1  0  0  0  0  0  0  0   0   0   0
537  0 

In [ ]:
# Calculate KLD
import scipy
import numpy
import pandas as pd
import math
base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\human_eval\\Mixtral-8x7B-Instruct-v0.1-bnb-4bit\\Chinese\\mistral\\"

final_data = pd.read_csv(base+"final_score.csv", index_col=0).fillna(0)[1:]
p_model_list = []

for i in final_data.index:
    if i != 0:
        line = final_data.loc[i]
        # print(line)
        prob_line = line / line.sum()
        p_model_list.append(prob_line)
# print(p_model_list)
# D:\projects\LLMValueInvestigation\code\human_evaluation\survey_data\AECE_v1\preferences
ref_data = pd.read_csv(r"F:\Projects\LLMValueInvestigation\code\human_evaluation\survey_data\AECE_v1\preferences\zh_ref_score_all.csv", index_col=0)
ref_data = ref_data.fillna(0)
    
question_type = pd.read_excel(
    r"F:\Projects\LLMValueInvestigation\code\human_evaluation\survey_data\AECE_v1\Originals\CN_survey\result_database\Question_type.xlsx"
)
types = question_type["value type"]
kl_list = []
ques_num = []
p_models= []
for i in range(len(p_model_list)):
    p_model = p_model_list[i]/p_model_list[i].sum()
    p_ref = ref_data.loc[i+1]/ref_data.loc[i+1].sum()
    kl_i = scipy.stats.entropy(p_model, p_ref)
    # # print(p_model)
    # print(p_model_list[i])
    # # print(p_ref)
    # print(ref_data.loc[i+1])
    # print(kl_i)
    kl_list.append(kl_i)
    p_models.append(p_model)
    ques_num.append(i+1)


kl_data = pd.DataFrame({"question Number": ques_num, "kl divergence": kl_list,
                        "type": types[:len(ques_num)]
                       })
dmax = 0
close_count = 0
for i in range(len(kl_data)):
    if kl_data.loc[i]["kl divergence"] < 0.5:
        close_count += 1
    if kl_data.loc[i]["kl divergence"] == numpy.inf:
        continue
    elif kl_data.loc[i]["kl divergence"] > dmax:
        dmax = kl_data.loc[i]["kl divergence"]
if numpy.nan in kl_data:
    print(a)
print(close_count)
print(dmax)
kl_data = kl_data.replace(numpy.inf, dmax+1)
kl_data = kl_data.replace(numpy.nan, dmax+1)
print(f"Mean: {numpy.mean(kl_data['kl divergence'])}")
print(f"STD: {numpy.mean(numpy.std(p_models, axis=1))}")
kl_data.to_csv(f"{base}Mean-{numpy.mean(kl_data['kl divergence'])}-qbq_kld.csv", index=True)

In [ ]:
# Diversity
qa_dict = {}
for j in range(100):
    qa_dict[j] = []
for i in range(19):
    with open(f"D:\\projects\\LLMValueInvestigation\\code\\investigate\\outputs\\DEF-wo_ppt\\Chinese\\dolphin-2.2.1-mistral-7b\\{i}\\0_test_responses.txt", 
              "r", encoding="utf-8") as ans:
        answers_raw = ans.read()
        answers_raw = answers_raw.split("\n")
        
    answers = []
    
    for i in range(len(answers_raw)):
        if answers_raw[i].startswith("Q"):
            answer = answers_raw[i]
            answers.append(answer)
    
        elif answers_raw != "\n":
            answers[-1] += answers_raw[i]
            
    # assert len(answers) == len(choices_list), f"length error: ans {len(answers)}, ques {len(choices_list)}"
    for j in range(100):
        qa_dict[j].append(answers[j])

un_count = 0
for k in range(100):
    qa_cleaned = list(set(qa_dict[k]))
    un_count += len(qa_cleaned)
print(un_count-100)

In [ ]:
from math import log2
# Case KL-D Single
import scipy
import numpy
import pandas as pd
base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\outputs\\Mixtral-8x7B-Instruct-v0.1-bnb-4bit\\Chinese\\mistral\\"
ref_data = pd.read_csv(r"F:\Projects\LLMValueInvestigation\code\human_evaluation\survey_data\AECE_v1\preferences\zh_ref_score_all.csv", index_col=0)
ref_data = ref_data.fillna(0)

mean_list = []
for i in range(20):
    final_data = pd.read_csv(base + str(i)+"\\qa_score_single.csv", index_col=0)
    p_model_list = []
    for i in final_data.index:
        if i != 0:
            line = final_data.loc[i]
            prob_line = line / line.sum()
            p_model_list.append(prob_line)
    # print(p_model_list)
    
    # calculate the kl divergence
    def kl_divergence(p, q):
     return sum(p[i] * log2(p[i]/q[i]) for i in range(len(p)))
        
    question_type = pd.read_excel(
        r"F:\Projects\LLMValueInvestigation\code\human_evaluation\survey_data\AECE_v1\Originals\CN_survey\result_database\Question_type.xlsx"
    )
    types = question_type["value type"]
    kl_list = []
    ques_num = []
    for i in range(len(p_model_list)):
        p_model = p_model_list[i]/p_model_list[i].sum()
        p_ref = ref_data.loc[i+1]/ref_data.loc[i+1].sum()
        kl_i = scipy.stats.entropy(p_model, p_ref)
        # print(kl_i)
        kl_list.append(kl_i)
        ques_num.append(i+1)
    
    
    kl_data = pd.DataFrame({"question Number": ques_num, "kl divergence": kl_list, "type": types})
    dmax = 0
    close_count = 0
    for i in range(len(kl_data)):
        if kl_data.loc[i]["kl divergence"] < 0.5:
            close_count += 1
        if kl_data.loc[i]["kl divergence"] == numpy.inf:
            continue
        elif kl_data.loc[i]["kl divergence"] > dmax:
            dmax = kl_data.loc[i]["kl divergence"]
    if numpy.nan in kl_data:
        print(a)
    kl_data = kl_data.replace(numpy.inf, dmax+1)
    kl_mean = kl_data["kl divergence"].mean()
    mean_list.append(kl_mean)
print(sorted(range(len(mean_list)), key=lambda k: mean_list[k]))

In [ ]:
# Single Character
model = "Mixtral-8x7B-Instruct-v0.1-bnb-4bit"
model_short = "mistral"
test_type = "CN_survey"
lang = "Chinese"
lang_short = "zh"
target = "5"

data_base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE_v1\\preferences\\"
var_chara_data = pd.read_excel(rf"F:\Projects\LLMValueInvestigation\code\human_evaluation\survey_data\AECE_v1\Originals\{test_type}\Character.xlsx", index_col=[0, 1])
base = f"F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\outputs\\{model}\\{lang}\\{model_short}"
human_res_data = pd.read_csv(f"{data_base}{lang_short}_ref_score_all.csv")[1:].drop('12', axis=1)
model_res_data = pd.read_csv(f"{base}\\{target}\\qa_score_single.csv", index_col=0)[1:].drop('12', axis=1)

profile_data = pd.DataFrame(columns=["Match", "Sex", "Age"], index=range(1,614))
multiple_choice_ques = [7, 8, 227, 228]

for i in range(1, len(model_res_data) + 1):
    # print(i)
    model_prefer = model_res_data.loc[i].idxmax()
    human_prefer = human_res_data.loc[i].idxmax()
    if model_prefer == human_prefer:
        profile_data.loc[i]["Match"] = "Y"
    else:
        profile_data.loc[i]["Match"] = "N"
    if i not in multiple_choice_ques:
        # print(var_chara_data.index)
        if (str(i), float(model_prefer)) in var_chara_data.index:
            profile_data.loc[i]["Sex"] = var_chara_data.loc[str(i), float(model_prefer)]["Sex"]
            profile_data.loc[i]["Age"] = var_chara_data.loc[str(i), int(model_prefer)]["Age"]
        else:
            profile_data.loc[i]["Sex"] = "Null"
            profile_data.loc[i]["Age"] = "Null"
    else:
        if (f"{str(i)}-{int(model_prefer)}", 1.0) in var_chara_data.index:
            profile_data.loc[i]["Sex"] = var_chara_data.loc[f"{str(i)}-{int(model_prefer)}", 1.0]["Sex"]
            profile_data.loc[i]["Age"] = var_chara_data.loc[f"{str(i)}-{int(model_prefer)}", 1.0]["Age"]
        else:
            profile_data.loc[i]["Sex"] = "Null"
            profile_data.loc[i]["Age"] = "Null"
        # print(profile_data.loc[i])
chara_base = r"F:\Projects\LLMValueInvestigation\code\human_evaluation\survey_data\AECE_v1\Originals"
question_type = pd.read_excel(
    f"{chara_base}\\{test_type}\\result_database\\Question_type.xlsx"
)
types = question_type["value type"]
types.index = range(1, len(types) + 1)
profile_data = pd.concat([profile_data, types], axis=1,)
# profile_data
profile_data.to_excel(f"{base}\\{target}\\Character.xlsx")

In [ ]:
# QBQ KL-D
import pandas as pd
import seaborn as sns

sns.set_style("white")
sns.set_context('paper')

# model_names = ["baichuan2-13B-4bits","WizardLM-13B-V1.2","ChatGLM2-6B", "dolphin-2.2.1-mistral-7b", "Meta-Llama-3-8B-Instruct", "Llama3-Chinese-8B-Instruct"]
model_name = "Mixtral-8x7B-Instruct-v0.1-bnb-4bit"
cn_base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\human_eval\\CN_survey_all\\" + model_name + "\\"
us_base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\human_eval\\US_survey_all\\" + model_name + "\\"
cn_kld_data = pd.read_csv(cn_base+"qbq_kld.csv", index_col=0)
us_kld_data = pd.read_csv(us_base+"qbq_kld.csv", index_col=0)

cn_test_type = pd.DataFrame(["CN Survey"]*len(cn_kld_data), index=range(len(cn_kld_data)), columns=["Test type"])
us_test_type = pd.DataFrame(["US Survey"]*len(us_kld_data), index=range(len(us_kld_data)), columns=["Test type"])

cn_kld = pd.concat([cn_kld_data, cn_test_type], axis=1)
us_kld = pd.concat([us_kld_data, us_test_type], axis=1)
kl_data = pd.concat([us_kld, cn_kld], axis=0, ignore_index=True)
kl_data.columns = ["Num question", "KL-Div", "Value type", "Test type"]
print(kl_data)
ax = sns.violinplot(data=kl_data, x="Value type", y="KL-Div", hue="Test type", split=True, palette="coolwarm")
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
barplt = ax.get_figure()
barplt.savefig(f"{model_name}-qbq.pdf",  bbox_inches='tight')

In [ ]:
# Value type KL-D
import pandas as pd
import seaborn as sns

model_name = ["Baichuan2-13B-Chat","WizardLM-13B","ChatGLM2-6B", "Mistral-7B-Instruct","Dolphin-2.2.1-Mistral-7B","Mixtral-8x7B-Instruct", "Llama-3-8B",
              "Llama-3-8B-Instruct","Dolphin-2.9.1-Llama-3-8B", "Llama-3-Chinese-8B-Instruct", "Claude-3.5-Sonnet"]

mean_data = pd.DataFrame()
overall_mean = {}
for i in range(len(model_name)):
    name = model_name[i]
    cn_base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\human_eval\\CN_survey_all\\" + name + "\\"
    cn_kld_data = pd.read_csv(cn_base+"qbq_kld.csv", index_col=0)
    cn_test_type = pd.DataFrame(["CN Survey"]*len(cn_kld_data), index=range(len(cn_kld_data)), columns=["Test type"])
    cn_kld = pd.concat([cn_kld_data, cn_test_type], axis=1)
    cn_kld.columns = ["Num question", "KL-Div", "Value type", "Test type"]
    
    cn_means = cn_kld.groupby('Value type')['KL-Div'].mean()
    mean = cn_means.mean()
    overall_mean[name] = mean
    temp = {
        "Value type": cn_means.index,
        "Mean KL-D": cn_means.values,
        "Model": [name] * len(cn_means)
    }
    
    mean_temp = pd.DataFrame(temp)
    mean_data = pd.concat([mean_data, mean_temp], ignore_index=True)
print(sorted(overall_mean.items(), key=lambda x: x[1], reverse=True))

In [ ]:
import pandas as pd
import os

base = "D:\\projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE_v1\\Originals\\CN_survey\\supplyment data"

css_data = pd.read_csv(f"{base}\\CGSS_female.csv")
wvs_data = pd.read_csv(f"{base}\\WVS_female.csv")
out_data = pd.concat([wvs_data, css_data], axis=1)
out_data = pd.concat([out_data, out_data], axis=1)
out_data.to_csv("female.csv", index=True)

In [ ]:
# Gather Character data
with open("D:\\projects\\LLMValueInvestigation\\Chinese_questionaire.txt", "r", encoding='utf-8') as q_f:
    questions = q_f.read()
    ques_list = questions.split('\n')

with open("D:\\projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE\\Chinese_questionaires.txt", "r", encoding='utf-8') as o_f:
    ori_ques = o_f.read()
    ori_list = ori_ques.split("\n")

mode = "50"
index_list = []
for ques in ques_list:
    index_list.append(ori_list.index(ques))
indexs = [ i+1 for i in index_list]

chara = pd.read_csv(
    f"D:\\projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE_v1\\Originals\\CN_survey\\supplyment data\\Character\\{mode}.csv", 
    index_col=0)

extra_list = []
for ind in indexs:
    if ind not in range(210, 215):
        extra_list.append(chara[str(ind)].values)
    else:
        length = len(chara[f"{str(ind)}-1"].values)
        extra_list.append([100]*length)

extr_data = pd.DataFrame(extra_list, index=range(441, 440 + len(extra_list) + 1)).T

final_chara = pd.concat([chara, extr_data], axis=1)
final_chara.to_excel(f"final_{mode}.xlsx", index=True)

In [ ]:
# Load human preference data
import pandas as pd
import numpy as np

base = "D:\\projects\\LLMValueInvestigation\\code\\human_evaluation\\human_eval\\"
test_type = "US_survey"
data_base = "D:\\projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE_v1\\preferences\\usa_"

chara_base = "D:\\projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE_v1\\Originals"
all_data = pd.read_csv(f"{chara_base}\\{test_type}\\Characters\\final_all.csv", dtype=np.float32)

male_data = pd.read_csv(f"{chara_base}\\{test_type}\\Characters\\final_male.csv",  dtype=np.float32)
female_data = pd.read_csv(f"{chara_base}\\{test_type}\\Characters\\final_female.csv", dtype=np.float32)

_29_data = pd.read_csv(f"{chara_base}\\{test_type}\\Characters\\final_29.csv", dtype=np.float32)
_30_data = pd.read_csv(f"{chara_base}\\{test_type}\\Characters\\final_30.csv", dtype=np.float32)
_50_data = pd.read_csv(f"{chara_base}\\{test_type}\\Characters\\final_50.csv", dtype=np.float32)

In [ ]:
# Prepare Human Character
vars = all_data.columns

def process_dis(data, v):
    _var = data.loc[:, v]
    # _var = _var[~_var.isin(["iap"])]
    _count = _var.value_counts()
    return _count.sort_index()

def get_count(counts, id):
    if id in counts.index:
        return counts[id]
    else:
        return 0

var_chara = []
for var in vars:
    all_count = process_dis(all_data, var)
    male_count = process_dis(male_data, var)
    female_count = process_dis(female_data, var)

    _29_count = process_dis(_29_data, var)
    _30_count = process_dis(_30_data, var)
    _50_count = process_dis(_50_data, var)

    choice_list =pd.DataFrame(columns=["Sex", "Age"], index=all_count.index)
    for ind in all_count.index:
        if ind >= 0:
            
            m_count = get_count(male_count, ind)
            f_count = get_count(female_count, ind)
            _2_count = get_count(_29_count, ind)
            _3_count = get_count(_30_count, ind)
            _5_count = get_count(_50_count, ind)
            
            if m_count > f_count:
                choice_list.loc[ind]["Sex"] = "M"
            elif f_count > m_count:
                choice_list.loc[ind]["Sex"] = "F"
            else:
                choice_list.loc[ind]["Sex"] = "T"
     
            if len(set([_2_count, _3_count, _5_count])) == 3:
                if max(_2_count, _3_count, _5_count) == _2_count:
                    choice_list.loc[ind]["Age"] = "29"
                elif max(_2_count, _3_count, _5_count) == _3_count:
                    choice_list.loc[ind]["Age"] = "30"
                elif max(_2_count, _3_count, _5_count) == _5_count:
                    choice_list.loc[ind]["Age"] = "50"
            elif _2_count == _3_count and max(_2_count, _3_count, _5_count) == _2_count:
                choice_list.loc[ind]["Age"] = "29-30"
            elif _2_count == _5_count and max(_2_count, _3_count, _5_count) == _2_count:
                choice_list.loc[ind]["Age"] = "29-50"
            elif _5_count == _3_count and max(_2_count, _3_count, _5_count) == _5_count:
                choice_list.loc[ind]["Age"] = "30-50"
            else:
                choice_list.loc[ind]["Age"] = "T"
            # print(choice_list)
    var_chara.append(choice_list.dropna())

In [ ]:
var_chara_data = pd.concat(var_chara, keys=[index.index.name for index in var_chara])
var_chara_data.to_excel(f"{chara_base}\\{test_type}\\Character.xlsx", index=True)

In [ ]:
# PCA and CulMap
def prepare_pca_data(theme_ques, final_data):
    theme_mean_list = []
    for theme in theme_ques:
        theme_sum = 0
        theme_count = 0
        for ques in theme:
            if ques in final_data.index:
                if ques not in [7,8] and final_data.loc[ques].sum() != 0:
                    q_mean = 0
                    for choice in final_data.loc[ques].index:
                        if choice != "12":
                            q_mean += float(choice) * (final_data.loc[ques][choice] / final_data.loc[ques].sum())
                    theme_sum += q_mean
                    theme_count+=1
        # print(f"final_data.loc[ques]: {final_data.loc[ques]}")
        # print(f"ques: {ques}")
        if theme_count != 0:
            theme_mean = theme_sum / theme_count
        else:
            theme_mean = 0
        theme_mean_list.append(theme_mean)
    return theme_mean_list

from sklearn.decomposition import PCA
import pandas as pd
import numpy

model_name = ["Baichuan2-13B-Chat","WizardLM-13B","ChatGLM2-6B", "Mistral-7B-Instruct","Dolphin-2.2.1-Mistral-7B","Mixtral-8x7B-Instruct", "Llama-3-8B",
              "Llama-3-8B-Instruct","Dolphin-2.9.1-Llama-3-8B", "Llama-3-Chinese-8B-Instruct", "Claude-3.5-Sonnet"]

features = []
feature_name = []

cn_question_type = pd.read_excel(
        "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE_v1\\Originals\\CN_survey\\result_database\\Question_type.xlsx")
type_index = cn_question_type.groupby("value type")
cn_theme_index = []
for index, value in type_index:
    cn_theme_index.append(value["id"].values.tolist())

cn_question_tp = cn_question_type
type_id = cn_question_tp.groupby("value type")
cn_theme_id = []
for index, value in type_id:
    cn_theme_id.append(value["id"].values.tolist())


us_question_type = pd.read_excel(
    "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE_v1\\Originals\\US_survey\\result_database\\Question_type.xlsx")
type_index = us_question_type.groupby("value type")
us_theme_index = []
for index, value in type_index:
    us_theme_index.append(value["id"].values.tolist())

us_question_tp = us_question_type
type_id = us_question_tp.groupby("value type")
us_theme_id = []
for index, value in type_id:
    us_theme_id.append(value["id"].values.tolist())

    

zh_ref_data = pd.read_csv(
        "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE_v1\\preferences\\zh_ref_score_all.csv",
        index_col=0).fillna(0)
en_ref_data = pd.read_csv(
    "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE_v1\\preferences\\usa_ref_score_all.csv", 
    index_col=0).fillna(0)

zh_ref_theme_mean = prepare_pca_data(theme_ques=cn_theme_index, final_data=zh_ref_data)
en_ref_theme_mean = prepare_pca_data(theme_ques=us_theme_index, final_data=en_ref_data)
features.append(en_ref_theme_mean)
features.append(zh_ref_theme_mean)
feature_name.append("Reference US")
feature_name.append("Reference CN")

for name in model_name:
    cn_score_base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\human_eval\\CN_survey_all\\"
    us_score_base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\human_eval\\US_survey_all\\"
    zh_final = pd.read_csv(cn_score_base + name + "\\final_score.csv", index_col=0)
    en_final = pd.read_csv(us_score_base + name + "\\final_score.csv", index_col=0)
    # if "Llama" in name:
    #     cn_theme_index = cn_theme_id
    #     us_theme_index = us_theme_id
    print(f"name: {name}")
    zh_theme_mean = prepare_pca_data(theme_ques=cn_theme_index, final_data=zh_final)
    en_theme_mean = prepare_pca_data(theme_ques=us_theme_index, final_data=en_final)
    features.append(en_theme_mean)
    feature_name.append(f"{name} US")  
    features.append(zh_theme_mean)
    feature_name.append(f"{name} CN")  

A = numpy.array(features)
pca = PCA(n_components=2)
pca.fit(A)
result = pca.transform(A)

pca_datapoint = {
    "Survival v.s. Self-Expression": result[:,0],
    "Traditional v.s. Secular": result[:,1],
    "Model type": feature_name
}
cord = pd.DataFrame(pca_datapoint, index=feature_name)

In [ ]:
c1 = sns.color_palette("husl", as_cmap=True,).resampled(12).colors # CN
c2 = sns.color_palette("hls", as_cmap=True,).resampled(12).colors # US
color_list = []
for cc1, cc2 in zip(c1, c2):
    color_list.append(cc2)
    color_list.append(cc1)
mixed_pal = sns.color_palette(color_list)
mixed_pal

In [ ]:
import seaborn as sns
import random

sns.set_context("paper")
p1 = sns.scatterplot(
    x=cord["Survival v.s. Self-Expression"], 
    y=cord["Traditional v.s. Secular"], 
    data=cord, hue=cord["Model type"], 
    palette=mixed_pal,sizes=[100]
)

for line in range(0,cord.shape[0]):
    if "CN" in cord.index[line] or "Reference" in cord.index[line]:
         p1.text(
             cord["Survival v.s. Self-Expression"][line],
             cord["Traditional v.s. Secular"][line] + random.uniform(-0.1, 0.1),
             cord.index[line], horizontalalignment='center', size='x-small', color='black'
         )
        
model_name = ["Reference", "Baichuan2-13B-Chat","WizardLM-13B","ChatGLM2-6B", "Mistral-7B-Instruct","Dolphin-2.2.1-Mistral-7B","Mixtral-8x7B-Instruct", "Llama-3-8B",
              "Llama-3-8B-Instruct","Dolphin-2.9.1-Llama-3-8B", "Llama-3-Chinese-8B-Instruct", "Claude-3.5-Sonnet"]

colors= ["#e96870", "#df9545", "#bfbb44", "#87c144","#45c667", "#47c5a9", "#46aec6", "#4888d8", "#8e75e8", "#d763e8", "#e85eb1", "#e86671"]
custom = sns.color_palette(colors)

line_dict = {
    "x": [],
    "y": [],
    "Model type": [],
}
for name in model_name:
    cn_cord = cord[cord["Model type"] == f"{name} CN"]
    from_x = cn_cord["Survival v.s. Self-Expression"].item()
    from_y = cn_cord["Traditional v.s. Secular"].item()
    line_dict["x"].append(from_x)
    line_dict["y"].append(from_y)
    line_dict["Model type"].append(name)
    us_cord = cord[cord["Model type"] == f"{name} US"]
    to_x = us_cord["Survival v.s. Self-Expression"].item()
    to_y = us_cord["Traditional v.s. Secular"].item()
    line_dict["x"].append(to_x)
    line_dict["y"].append(to_y)
    line_dict["Model type"].append(name)
    
line_data = pd.DataFrame(line_dict)
sns.lineplot(line_data, x="x", y="y", hue="Model type", palette=custom, linestyle='-.')
sns.move_legend(p1, "upper left", bbox_to_anchor=(1, 1), ncols=2)
dot_fig = p1.get_figure()
dot_fig.savefig("culmap_labels.pdf", bbox_inches='tight')

In [ ]:
# Profile Drawing for Model
data_base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE_v1\\preferences\\"
name = "Mixtral-8x7B-Instruct-v0.1-bnb-4bit"
test_type = "CN_survey"
lang_short = "zh"

var_chara_data = pd.read_excel(f"F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE_v1\\Originals\\{test_type}\\Character.xlsx", index_col=[0, 1])
base = f"F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\human_eval\\{test_type}_all"
human_res_data = pd.read_csv(f"{data_base}{lang_short}_ref_score_all.csv")[1:].drop('12', axis=1)
model_res_data = pd.read_csv(f"{base}\\{name}\\final_score.csv", index_col=0)[1:].drop('12', axis=1)

profile_data = pd.DataFrame(columns=["Match", "Sex", "Age"], index=range(1,614))
multiple_choice_ques = [7, 8, 227, 228]

for i in range(1, len(model_res_data) + 1):
    # print(i)
    model_prefer = model_res_data.loc[i].idxmax()
    human_prefer = human_res_data.loc[i].idxmax()
    if model_prefer == human_prefer:
        profile_data.loc[i]["Match"] = "Y"
    else:
        profile_data.loc[i]["Match"] = "N"
    if i not in multiple_choice_ques:
        # print(var_chara_data.index)
        if (str(i), float(model_prefer)) in var_chara_data.index:
            profile_data.loc[i]["Sex"] = var_chara_data.loc[str(i), float(model_prefer)]["Sex"]
            profile_data.loc[i]["Age"] = var_chara_data.loc[str(i), int(model_prefer)]["Age"]
        else:
            profile_data.loc[i]["Sex"] = "Null"
            profile_data.loc[i]["Age"] = "Null"
    else:
        if (f"{str(i)}-{int(model_prefer)}", 1.0) in var_chara_data.index:
            profile_data.loc[i]["Sex"] = var_chara_data.loc[f"{str(i)}-{int(model_prefer)}", 1.0]["Sex"]
            profile_data.loc[i]["Age"] = var_chara_data.loc[f"{str(i)}-{int(model_prefer)}", 1.0]["Age"]
        else:
            profile_data.loc[i]["Sex"] = "Null"
            profile_data.loc[i]["Age"] = "Null"
        # print(profile_data.loc[i])
chara_base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE_v1\\Originals"
question_type = pd.read_excel(
    f"{chara_base}\\{test_type}\\result_database\\Question_type.xlsx"
)
types = question_type["value type"]
types.index = range(1, len(types) + 1)
profile_data = pd.concat([profile_data, types], axis=1,)
# profile_data
profile_data.to_excel(f"{base}\\{name}\\Character.xlsx")

In [ ]:
# Matching Profile barplot
import seaborn as sns
base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\human_eval\\"
test_type = "US_survey_all"

model_name = ["Baichuan2-13B-Chat","WizardLM-13B","ChatGLM2-6B", "Mistral-7B-Instruct","Dolphin-2.2.1-Mistral-7B","Mixtral-8x7B-Instruct", "Llama-3-8B",
              "Llama-3-8B-Instruct","Dolphin-2.9.1-Llama-3-8B", "Llama-3-Chinese-8B-Instruct", "Claude-3.5-Sonnet"]

v_match_list = []
model_list = []
v_list = []
vs = ["EVI", "IDV", "IVR", "LTO","MAS","PDI","TDI","UAI","USI"]
for name in model_name:
    m_chara = pd.read_excel(f"{base}{test_type}\\{name}\\Character.xlsx", index_col=0)
    m_chara_v = m_chara.groupby("value type")
    for v, m_c_v in m_chara_v:
        model_list.append(name)
        v_list.append(v)
        if "Y" in m_c_v["Match"].value_counts().index:
            v_match_list.append(m_c_v["Match"].value_counts()["Y"]/len(m_c_v["Match"])*100)
        else:
            v_match_list.append(0)

cn_overall_match = {
    "Model": model_list,
    "Precentage of match": v_match_list,
    # "Mismatch": [540]*len(model_list),
    "Value type": v_list
}
cn_overall_match_data = pd.DataFrame(cn_overall_match)
s2 = sns.barplot(data=cn_overall_match_data, x="Value type", y="Precentage of match", hue="Model", palette="tab20c")
sns.move_legend(s2, "upper left", bbox_to_anchor=(1, 1))
barfig = s2.get_figure()
barfig.savefig(f"{test_type}_matching.pdf", bbox_inches='tight')

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as matlib

sns.set()
sns.set_context('paper')
# sns.set_style("whitegrid")
of_base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\human_eval\\overall_profile.csv"
overall_profiles = pd.read_csv(of_base)
model_name = ["Mixtral-8x7B-Instruct"]

m_o_p = overall_profiles.groupby("Model")

for m, overall_profile in m_o_p:
    
    mat_overall = overall_profile[overall_profile["Profile"]=="matching"]
    mat_sex = mat_overall[mat_overall["Character type"]=="sex"]
    
    mat_us_total = mat_sex[mat_sex["Test type"]=="US survey"]["Number"].sum()
    mat_us_male = mat_sex[mat_sex["Test type"]=="US survey"][mat_sex["Character"]=="Male"]["Number"].tolist()[0]
    mat_us_female = mat_sex[mat_sex["Test type"]=="US survey"][mat_sex["Character"]=="Female"]["Number"].tolist()[0]
    
    mat_cn_total = mat_sex[mat_sex["Test type"]=="CN survey"]["Number"].sum()
    mat_cn_male = mat_sex[mat_sex["Test type"]=="CN survey"][mat_sex["Character"]=="Male"]["Number"].tolist()[0]
    mat_cn_female = mat_sex[mat_sex["Test type"]=="CN survey"][mat_sex["Character"]=="Female"]["Number"].tolist()[0]
    
    mis_overall = overall_profile[overall_profile["Profile"]=="mismatch"]
    mis_sex = mis_overall[mis_overall["Character type"]=="sex"]
    
    mis_us_total = mis_sex[mis_sex["Test type"]=="US survey"]["Number"].sum()
    mis_us_male = mis_sex[mis_sex["Test type"]=="US survey"][mis_sex["Character"]=="Male"]["Number"].tolist()[0]
    mis_us_female = mis_sex[mis_sex["Test type"]=="US survey"][mis_sex["Character"]=="Female"]["Number"].tolist()[0]
    
    mis_cn_total = mis_sex[mis_sex["Test type"]=="CN survey"]["Number"].sum()
    mis_cn_male = mis_sex[mis_sex["Test type"]=="CN survey"][mis_sex["Character"]=="Male"]["Number"].tolist()[0]
    mis_cn_female = mis_sex[mis_sex["Test type"]=="CN survey"][mis_sex["Character"]=="Female"]["Number"].tolist()[0]
    
    plt.figure(figsize=(2,1))
    fig,axes=plt.subplots(1,2)
    
    mat_total_data = pd.DataFrame({
        "Test type": ["US survey", "CN survey"],
        "Percentage": [mat_us_total/mat_us_total*100, mat_cn_total/mat_cn_total*100]
    })
    
    mat_top_plt = sns.barplot(data=mat_total_data, x="Test type", y="Percentage", color="#9BB5F0", ax=axes[0])
    
    mat_bottom_data = pd.DataFrame({
        "Test type": ["US survey", "CN survey"],
        "Percentage": [mat_us_female/mat_us_total*100, mat_cn_female/mat_cn_total*100]
    })
    
    mat_bottom_plt = sns.barplot(data=mat_bottom_data, x="Test type", y="Percentage", color="#E5A089", ax=axes[0])
    
    mis_total_data = pd.DataFrame({
        "Test type": ["US survey", "CN survey"],
        "Percentage": [mis_us_total/mis_us_total*100, mis_cn_total/mis_cn_total*100]
    })
    
    mis_top_plt = sns.barplot(data=mis_total_data, x="Test type", y="Percentage", color="#9BB5F0",  ax=axes[1])
    
    mis_bottom_data = pd.DataFrame({
        "Test type": ["US survey", "CN survey"],
        "Percentage": [mis_us_female/mis_us_total*100, mis_cn_female/mis_cn_total*100]
    })
    
    mis_bottom_plt = sns.barplot(data=mis_bottom_data, x="Test type", y="Percentage", color="#E5A089",  ax=axes[1])
    
    axes[0].set_xlabel("Match")
    axes[1].set_xlabel("Mismatch")
    
    axes[0].set_facecolor("1")
    axes[1].set_facecolor(".95")
    
    topbar = matlib.patches.Rectangle((0,0),1,1,fc="#9BB5F0", edgecolor = 'none')
    bottombar = matlib.patches.Rectangle((0,0),1,1,fc='#E5A089',  edgecolor = 'none')
    mis_bottom_plt.legend(handles=[topbar, bottombar], labels=['Male', 'Female'], 
                          loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, facecolor="white", title="Sex")
    plt.savefig(f"{m}_overall_profile_sex.pdf", bbox_inches='tight')

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as matlib

sns.set()
sns.set_context('paper')
# sns.set_style("whitegrid")
of_base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\human_eval\\overall_profile.csv"
overall_profiles = pd.read_csv(of_base)
model_name = ["Claude-3.5-Sonnet"]

m_o_p = overall_profiles.groupby("Model")

for m, overall_profile in m_o_p:
    mat_overall = overall_profile[overall_profile["Profile"]=="matching"]
    mat_age = mat_overall[mat_overall["Character type"]=="age"]
    
    mat_us_total = mat_age[mat_age["Test type"]=="US survey"]["Number"].sum()
    mat_us_29 = mat_age[mat_age["Test type"]=="US survey"][mat_age["Character"]=="Up to 29"]["Number"].tolist()[0]
    mat_us_30 = mat_age[mat_age["Test type"]=="US survey"][mat_age["Character"]=="30 to 49"]["Number"].tolist()[0]
    mat_us_50 = mat_age[mat_age["Test type"]=="US survey"][mat_age["Character"]=="Over 50"]["Number"].tolist()[0]
    
    mat_cn_total = mat_age[mat_age["Test type"]=="CN survey"]["Number"].sum()
    mat_cn_29 = mat_age[mat_age["Test type"]=="CN survey"][mat_age["Character"]=="Up to 29"]["Number"].tolist()[0]
    mat_cn_30 = mat_age[mat_age["Test type"]=="CN survey"][mat_age["Character"]=="30 to 49"]["Number"].tolist()[0]
    mat_cn_50 = mat_age[mat_age["Test type"]=="CN survey"][mat_age["Character"]=="Over 50"]["Number"].tolist()[0]
    
    mis_overall = overall_profile[overall_profile["Profile"]=="mismatch"]
    mis_age = mis_overall[mis_overall["Character type"]=="age"]
    
    mis_us_total = mis_age[mis_age["Test type"]=="US survey"]["Number"].sum()
    mis_us_29 = mis_age[mis_age["Test type"]=="US survey"][mis_age["Character"]=="Up to 29"]["Number"].tolist()[0]
    mis_us_30 = mis_age[mis_age["Test type"]=="US survey"][mis_age["Character"]=="30 to 49"]["Number"].tolist()[0]
    mis_us_50 = mis_age[mis_age["Test type"]=="US survey"][mis_age["Character"]=="Over 50"]["Number"].tolist()[0]
    
    mis_cn_total = mis_age[mis_age["Test type"]=="CN survey"]["Number"].sum()
    mis_cn_29 = mis_age[mis_age["Test type"]=="CN survey"][mis_age["Character"]=="Up to 29"]["Number"].tolist()[0]
    mis_cn_30 = mis_age[mis_age["Test type"]=="CN survey"][mis_age["Character"]=="30 to 49"]["Number"].tolist()[0]
    mis_cn_50 = mis_age[mis_age["Test type"]=="CN survey"][mis_age["Character"]=="Over 50"]["Number"].tolist()[0]
    
    plt.figure(figsize=(2.5,1))
    fig,axes=plt.subplots(1,2)
    
    mat_total_data = pd.DataFrame({
        "Test type": ["US survey", "CN survey"],
        "Percentage": [mat_us_total/mat_us_total*100, mat_cn_total/mat_cn_total*100]
    })
    mat_top_plt = sns.barplot(data=mat_total_data, x="Test type", y="Percentage", color="#9BB5F0", ax=axes[0])
    
    mat_middle_data = pd.DataFrame({
        "Test type": ["US survey", "CN survey"],
        "Percentage": [(mat_us_30+mat_us_50)/mat_us_total*100, (mat_cn_30+mat_cn_50)/mat_cn_total*100]
    })
    mat_middle_plt = sns.barplot(data=mat_middle_data, x="Test type", y="Percentage", color="#DDDCDC", ax=axes[0])
    
    mat_bottom_data = pd.DataFrame({
        "Test type": ["US survey", "CN survey"],
        "Percentage": [mat_us_50/mat_us_total*100, mat_cn_50/mat_cn_total*100]
    })
    mat_bottom_plt = sns.barplot(data=mat_bottom_data, x="Test type", y="Percentage", color="#E5A089", ax=axes[0])
    
    
    mis_total_data = pd.DataFrame({
        "Test type": ["US survey", "CN survey"],
        "Percentage": [mis_us_total/mis_us_total*100, mis_cn_total/mis_cn_total*100]
    })
    
    mis_top_plt = sns.barplot(data=mis_total_data, x="Test type", y="Percentage", color="#9BB5F0",  ax=axes[1])
    
    mis_middle_data = pd.DataFrame({
        "Test type": ["US survey", "CN survey"],
        "Percentage": [(mis_us_30+mis_us_50)/mis_us_total*100, (mis_cn_30+mis_cn_50)/mis_cn_total*100]
    })
    mis_middle_plt = sns.barplot(data=mis_middle_data, x="Test type", y="Percentage", color="#DDDCDC", ax=axes[1])
    
    mis_bottom_data = pd.DataFrame({
        "Test type": ["US survey", "CN survey"],
        "Percentage": [mis_us_50/mis_us_total*100, mis_cn_50/mis_cn_total*100]
    })
    mis_bottom_plt = sns.barplot(data=mis_bottom_data, x="Test type", y="Percentage", color="#E5A089", ax=axes[1])
    axes[0].set_xlabel("Match")
    axes[1].set_xlabel("Mismatch")
    axes[0].set_facecolor("1")
    axes[1].set_facecolor(".95")
    
    topbar = matlib.patches.Rectangle((0,0),1,1,fc="#9BB5F0", edgecolor = 'none')
    middlebar = matlib.patches.Rectangle((0,0),1,1,fc='#DDDCDC', edgecolor = 'none')
    bottombar = matlib.patches.Rectangle((0,0),1,1,fc='#E5A089',  edgecolor = 'none')
    mis_bottom_plt.legend(handles=[topbar, middlebar, bottombar], labels=['Up to 29', "30 to 49", 'Over 50'], 
                          loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, facecolor="white", title="Age")
    
    plt.savefig(f"{m}_overall_profile_age.pdf", bbox_inches='tight')

In [ ]:
import seaborn as sns
base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\human_eval\\"
test_type = "CN_survey_all"

model_name = ["Mistral-7B-Instruct","Mixtral-8x7B-Instruct", "Llama-3-8B",
              "Dolphin-2.9.1-Llama-3-8B", "Claude-3.5-Sonnet"]

v_match_list = []
model_list = []
v_list = []

def count(_count_, _count, _count_type, tie_, keys, va):
    if ('N', _count_type, keys, va) in _count.index:
        _count_ += _count[('N', _count_type, keys, va)]
    if ('N', 'T', keys, va) in tie_.index:
        _count_ += tie_[('N', 'T', keys, va)]
    return _count_

def gender_count(_count, type, tie, vb):
    _count_29 = 0
    _count_30 = 0
    _count_50 = 0
    for _29_key in _29_keys:
        _count_29 = count(_count_29, _count, type, tie, _29_key, vb)
    for _30_key in _30_keys:
        _count_30 = count(_count_30, _count, type, tie, _30_key, vb)
    for _50_key in _50_keys:
        _count_50 = count(_count_50, _count, type, tie, _50_key, vb)
    return _count_29, _count_30, _count_50

_29_keys = ["29", "29-30", "29-50", "T"]
_30_keys = ["30", "30-50", "29-30", "T"]
_50_keys = ["50", "30-50", "29-50", "T"]

value_list = []
sex_list = []
age_list = []
number_list = []
model_list = []

for name in model_name:
    m_chara = pd.read_excel(f"{base}{test_type}\\{name}\\Character.xlsx", index_col=0)
    m_chara = m_chara[m_chara["Match"]=="N"]
    m_chara_v = m_chara.groupby("value type")
    for v, m_c_v in m_chara_v:
        m_c_v_male = m_c_v[m_c_v["Sex"] == "M"]
        m_c_v_tie = m_c_v[m_c_v["Sex"] == "T"]
        m_c_v_female = m_c_v[m_c_v["Sex"] == "F"]
        
        male_count = m_c_v_male.value_counts()
        tie_count = m_c_v_tie.value_counts()
        female_count = m_c_v_female.value_counts()
        
        male_count_29, male_count_30, male_count_50 = gender_count(male_count, "M", tie_count, v)
        female_count_29, female_count_30, female_count_50 = gender_count(female_count, "F", tie_count, v)
        
        def store_line(va, sa, aa, na, ma):
            value_list.append(va)
            sex_list.append(sa)
            age_list.append(aa)
            number_list.append(na)
            model_list.append(ma)
        store_line(v, "Male", "29", male_count_29, name)
        store_line(v, "Male", "30", male_count_30, name)
        store_line(v, "Male", "50", male_count_50, name)
        
        store_line(v, "Female", "29", female_count_29, name)
        store_line(v, "Female", "30", female_count_30, name)
        store_line(v, "Female", "50", female_count_50, name)

mis_prof_data = pd.DataFrame({
    "Value type": value_list,
    "Sex": sex_list,
    "Age": age_list,
    "Number": number_list,
    "Model": model_list
})

mis_prof_data.to_excel(f"{base}{test_type}\\mis_prof_data.xlsx")

In [ ]:
# Mismatch profile character
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\human_eval\\"
test_type = "CN_survey_all"

overall_profile = pd.read_excel(f"{base}{test_type}\\mis_prof_data.xlsx", index_col=0)

topicss = [["EVI", "IDV", "IVR"], ["LTO","MAS","PDI"], ["TDI","UAI","USI"]]

model_name = ["Mistral-7B-Instruct","Mixtral-8x7B-Instruct", "Llama-3-8B",
              "Dolphin-2.9.1-Llama-3-8B", "Claude-3.5-Sonnet"]

for name in model_name:
    fig, ax =plt.subplots(3,3 ,figsize=(90,45))
    sns.set()
    sns.set_context('paper', font_scale=8)
    for j in range(len(topicss)):
        topics = topicss[j]
        for i in range(len(topics)):
            
            sample = overall_profile[overall_profile["Value type"]==topics[i]]
            sample = sample[sample["Model"]==name]
            
            heat_data = sample.pivot(index="Sex", columns="Age", values="Number")
            axesSub = sns.heatmap(heat_data, annot=True, cmap=sns.cubehelix_palette(as_cmap=True), ax=ax[j][i])
            # US color: start=.5, rot=-.5, as_cmap=True, CN color: as_cmap=True
            axesSub.set_title(f"Value type = {topics[i]}")
            axesSub.set_xlabel("")
            axesSub.set_ylabel("")
    plt.savefig(f"{test_type}_{name}_topical_profile_heat.pdf", bbox_inches='tight')
    

In [ ]:
# Value type KL-D lang abalation
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# model_name = ["Chinese", "en_Chinese"]
model_name = ["English", "zh_English"]

mean_data = pd.DataFrame()
overall_mean = []
for i in range(len(model_name)):
    name = model_name[i]
    cn_base = "D:\\projects\\LLMValueInvestigation\\code\\human_evaluation\\human_eval\\lang_abla\\" + name + "\\"
    cn_kld_data = pd.read_csv(cn_base+"qbq_kld.csv", index_col=0)
    cn_test_type = pd.DataFrame(["US Survey"]*len(cn_kld_data), index=range(len(cn_kld_data)), columns=["Test type"])
    cn_kld = pd.concat([cn_kld_data, cn_test_type], axis=1)
    cn_kld.columns = ["Num question", "KL-Div", "Value type", "Test type"]
    
    cn_means = cn_kld.groupby('Value type')['KL-Div'].mean()
    mean = cn_means.mean()
    overall_mean.append(mean)
    temp = {
        "Value type": cn_means.index,
        "Mean KL-D": cn_means.values,
        "Model": [name] * len(cn_means)
    }
    
    mean_temp = pd.DataFrame(temp)
    mean_data = pd.concat([mean_data, mean_temp])
# sns.set("white")
sns.set_style("white")
sns.set_context('paper')
# mean_data
ax = sns.barplot(data=mean_data, x="Value type", y="Mean KL-D", hue="Model", palette="coolwarm")
colors = ["#8BA5E9", "#EACDBE"]
for i in range(len(model_name)): 
    plt.axhline(y=overall_mean[i],ls="-.",c=colors[i])#添加水平直线
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
barplt = ax.get_figure()
barplt.savefig(f"us_val_mean_aba.pdf",  bbox_inches='tight')

In [ ]:
# PCA and CulMap
def prepare_pca_data(theme_ques, final_data):
    theme_mean_list = []
    for theme in theme_ques:
        theme_sum = 0
        theme_count = 0
        for ques in theme:
            if ques not in [7,8] and final_data.loc[ques].sum() != 0:
                q_mean = 0
                for choice in final_data.loc[ques].index:
                    if choice != "12":
                        q_mean += float(choice) * (final_data.loc[ques][choice] / final_data.loc[ques].sum())
                theme_sum += q_mean
                theme_count+=1
        theme_mean = theme_sum / theme_count
        theme_mean_list.append(theme_mean)
    return theme_mean_list

from sklearn.decomposition import PCA
import pandas as pd
import numpy

model_name = ["Chinese", "en_Chinese", "English", "zh_English"]
features = []
feature_name = []

cn_question_type = pd.read_excel(
        "D:\\projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE_v1\\Originals\\CN_survey\\result_database\\Question_type.xlsx")[:219]
type_index = cn_question_type.groupby("value type")
cn_theme_index = []
for index, value in type_index:
    cn_theme_index.append(value["id"].values.tolist())

us_question_type = pd.read_excel(
    "D:\\projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE_v1\\Originals\\US_survey\\result_database\\Question_type.xlsx")[:296]
type_index = us_question_type.groupby("value type")
us_theme_index = []
for index, value in type_index:
    us_theme_index.append(value["id"].values.tolist())

zh_ref_data = pd.read_csv(
        "D:\\projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE_v1\\preferences\\zh_ref_score_all.csv",
        index_col=0).fillna(0)
en_ref_data = pd.read_csv(
    "D:\\projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE_v1\\preferences\\usa_ref_score_all.csv", 
    index_col=0).fillna(0)

zh_ref_theme_mean = prepare_pca_data(theme_ques=cn_theme_index, final_data=zh_ref_data)
en_ref_theme_mean = prepare_pca_data(theme_ques=us_theme_index, final_data=en_ref_data)
features.append(en_ref_theme_mean)
features.append(zh_ref_theme_mean)
feature_name.append("Reference US")
feature_name.append("Reference CN")

for name in model_name:
    cn_score_base = "D:\\projects\\LLMValueInvestigation\\code\\human_evaluation\\human_eval\\lang_abla\\"
    us_score_base = "D:\\projects\\LLMValueInvestigation\\code\\human_evaluation\\human_eval\\lang_abla\\"
    if "Chinese" in name:
        zh_final = pd.read_csv(cn_score_base + name + "\\final_score.csv", index_col=0)
        zh_theme_mean = prepare_pca_data(theme_ques=cn_theme_index, final_data=zh_final)
        features.append(zh_theme_mean)
        feature_name.append(f"{name} CN")  
    elif "English" in name:
        en_final = pd.read_csv(us_score_base + name + "\\final_score.csv", index_col=0)
        en_theme_mean = prepare_pca_data(theme_ques=us_theme_index, final_data=en_final)
        features.append(en_theme_mean)
        feature_name.append(f"{name} US")  
    
A = numpy.array(features)
pca = PCA(n_components=2)
pca.fit(A)
result = pca.transform(A)

pca_datapoint = {
    "Survival v.s. Self-Expression": result[:,0],
    "Traditional v.s. Secular": result[:,1],
    "Model type": feature_name
}
cord = pd.DataFrame(pca_datapoint, index=feature_name)

In [ ]:
import seaborn as sns

sns.set_context("talk")
p1 = sns.scatterplot(
    x=cord["Survival v.s. Self-Expression"], 
    y=cord["Traditional v.s. Secular"], 
    data=cord, hue=cord["Model type"], 
    palette ='Paired',sizes=[100]
)
sns.move_legend(p1, "upper left", bbox_to_anchor=(1, 1))
# for line in range(0,cord.shape[0]):
#      p1.text(cord["Survival v.s. Self-Expression"][line], cord["Traditional v.s. Secular"][line]+0.1, cord.index[line], horizontalalignment='left', size='medium', color='black')
dot_fig = p1.get_figure()
dot_fig.savefig("culmap.pdf", bbox_inches='tight')

In [ ]:
# Insensitivity Measurement radar

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import pandas as pd

model_name = ["Mixtral-8x7B-Instruct"]
base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\gpt_judge\\"

for name in model_name:
    sns.set()
    plt.style.use('ggplot')
    data = pd.read_csv(f"{base}US_survey_all\\{name}\\AECE_score.csv", index_col=0)
    data = data.sum(axis=0)
    total_num = data.sum()
    
    zh_data = pd.read_csv(f"{base}CN_survey_all\\{name}\\AECE_score.csv", index_col=0)
    zh_data = zh_data.sum(axis=0)
    zh_total_num = zh_data.sum()
    
    suffi_list = [i for i in data[1:7]]
    zh_suffi_list = [i for i in zh_data[1:7]]
    
    # for i in data[1:7]:
    #     suffi = total_num - i
    #     suffi_list.append(suffi)
    
    labels = np.array(["IU", "QC", "RP", "CV", "FF", "IR"])
    # stats = [100.0, 95.0, 95.8, 62.8, 96.8]
    
    angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False)
    stats = np.concatenate((suffi_list, [suffi_list[0]]))
    zh_stats = np.concatenate((zh_suffi_list, [zh_suffi_list[0]]))
    
    angles = np.concatenate((angles, [angles[0]]))
    labels = np.concatenate((labels, [labels[0]]))
    
    fig = plt.figure()
    ax = fig.add_subplot(111, polar=True)
    ax.plot(angles, zh_stats, '.-', linewidth=1, label="CN")
    ax.plot(angles, stats, '.-', linewidth=1, label="US")
    
    ax.fill(angles, zh_stats, alpha=0.25)
    ax.fill(angles, stats, alpha=0.25)
    
    # # 设置中文字体
    # # font = FontProperties(fname=r"C:\Windows\Fonts\simhei.ttf", size=14)
    ax.set_thetagrids(angles * 180/np.pi, labels)
    
    # ax.set_ylim(0,5)
    plt.legend(loc='upper right', bbox_to_anchor=(1.2,1))
    ax.grid(True)
    p = ax.get_figure()
    p.savefig(f"{name}-radar.pdf", bbox_inches='tight')
    # plt.show()

In [ ]:
# Define a function to convert RGB values to a hexadecimal color code
def rgb_to_hex(r, g, b):
    # Use string formatting to convert the RGB values to a hexadecimal color code
    return ('{:02X}' * 3).format(r, g, b)
cs = sns.color_palette("tab20c", 11)
print(cs.as_hex())
cs.as_hex()
# for line in cs:
#     print(line[:-1])
#     co_hex = rgb_to_hex(line[0], line[1], line[2])
#     print(co_hex)
#     print(a)

In [ ]:
# Insensitivity Measurement radar
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import pandas as pd
import seaborn as sns

model_name = ["Baichuan2-13B-Chat","WizardLM-13B","ChatGLM2-6B", "Mistral-7B-Instruct","Dolphin-2.2.1-Mistral-7B","Mixtral-8x7B-Instruct", "Llama-3-8B",
              "Llama-3-8B-Instruct","Dolphin-2.9.1-Llama-3-8B", "Llama-3-Chinese-8B-Instruct", "Claude-3.5-Sonnet"]

base = "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\gpt_judge\\"
test_type = "US_survey_all"

line_style = ['-', '--', ':', '.-', '.-', '.-', '-*', '-*', '-*', '-*', 'o-']
cs = sns.color_palette("tab20c", 11)
cs_list = cs.as_hex()
datas = {}
for i in range(len(model_name)):
    name = model_name[i]
    bai_data = pd.read_csv(f"{base}{test_type}\\{name}\\AECE_score.csv", index_col=0)
    bai_data = bai_data.sum(axis=0)
    bai_suffi_list = [i for i in bai_data[1:7]]
    bai_stats = np.concatenate((bai_suffi_list, [bai_suffi_list[0]]))
    datas[name] = (bai_stats, line_style[i], cs_list[i])
    
plt.style.use('ggplot')

labels = np.array(["IU", "QC", "RP", "CV", "FF", "IR"])
angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False)
angles = np.concatenate((angles, [angles[0]]))
labels = np.concatenate((labels, [labels[0]]))

fig = plt.figure()
ax = fig.add_subplot(111, polar=True)

for model in datas.keys():
    ax.plot(angles, datas[model][0], datas[model][1], linewidth=1, label=model, color=datas[model][2])
    ax.fill(angles, datas[model][0], alpha=0.25, color=datas[model][2])
    # ax.grid(True)

ax.set_thetagrids(angles * 180/np.pi, labels)
plt.legend(loc='center left', bbox_to_anchor=(1.2, 0.8))
ax.grid(True)
p = ax.get_figure()
p.savefig(f"{test_type}-radar.pdf", bbox_inches='tight')
# plt.show()

In [ ]:
import pandas as pd
import random

test_types = ["CN_survey_all", "US_survey_all"]
model_names = ["baichuan2-13B-4bits", "ChatGLM2-6B", "dolphin-2.2.1-mistral-7b", "dolphin-2.9.1-llama-8b", "Llama3-Chinese-8B-Instruct", 
               "Meta-Llama-3-8B", "Meta-Llama-3-8B-Instruct", "Mistral-7B-Instruct-v0.1", "WizardLM-13B-V1.2"]
for test_type in test_types:
    sampled_sum = pd.Series([0]*13, index=[str(i) for i in range(0,13)])
    for name in model_names:
        data_path = rf"F:\Projects\LLMValueInvestigation\code\human_evaluation\gpt_judge\{test_type}\{name}\AECE_score.csv"
        data = pd.read_csv(data_path, index_col=0).drop(0)
        cleaned_data = data.loc[(data != 0).any(axis=1)]
        rand_index = random.sample(sorted(cleaned_data.index), 10)
        sampled_data = []
        for ind in rand_index:
            sampled_data.append(cleaned_data.loc[ind])
        sample_data = pd.concat(sampled_data, ignore_index=True, axis=1)
        sampled_sum = sampled_sum.radd(sample_data.sum(axis=1))
    sampled_sum.to_csv(f"{test_type}_sampled_sum.csv")

In [ ]:
import seaborn as sns
sns.set_context("paper")
sns.set_style("white")
cn_model = [27,20,31,34,47,34, 419]
cn_human = [45,24,33,48,43,59, 410]
us_model = [30,34,30,46,56,41, 387]
us_human = [55,43,49,59,50,68, 360]
cn_data = {
    "Insensitivity Measurement Dimensions": ["UI", "QC", "RP", "CV", "FF", "IR", "Pass"]*2,
    "Score": cn_model + cn_human,
    "From": ["GPT-3.5"]*len(cn_model) + ["Human"]*len(cn_human),
}
df = pd.DataFrame(data)
fig = sns.lineplot(cn_data, x="Insensitivity Measurement Dimensions", y="Score", hue="From", palette="YlOrBr")
p = fig.get_figure()
p.savefig("CN_judge_effect.pdf", bbox_inches='tight')

# us_data = {
#     "Insensitivity Measurement Dimensions": ["UI", "QC", "RP", "CV", "FF", "IR", "Pass"]*2,
#     "Score": us_model + us_human,
#     "From": ["GPT-3.5"]*len(cn_model) + ["Human"]*len(cn_human),
# }
# df = pd.DataFrame(data)
# fig = sns.lineplot(us_data, x="Insensitivity Measurement Dimensions", y="Score", hue="From", palette="Blues")
# p = fig.get_figure()
# p.savefig("US_judge_effect.pdf", bbox_inches='tight')

In [ ]:
import json
from typing import Dict, List
import pandas as pd

data_json_path = r"F:\Projects\EMNLP\llama3-main\llama3-main\data\sft_wvs_train.json"
with open(data_json_path, "r", encoding="utf-8")as f:
    data = json.load(f)

country_list = []
for d in data:
    instruct = d["instruction"]
    country_start = instruct.find("How would someone from ") + len("How would someone from ")
    country_end = instruct.find(" answer the following question:")
    country = instruct[country_start: country_end]
    country_list.append(country)
    
dis_data: Dict[str, List] = {k: [] for k in country_list}
for d in data:
    instruct = d["instruction"]
    country_start = instruct.find("How would someone from ") + len("How would someone from ")
    country_end = instruct.find(" answer the following question:")
    country = instruct[country_start: country_end]
    dist_list = list(d["options_dist"].values())
    dis_data[country].append(dist_list)

c_datas = []
for c in country_list:
    maxlen = max(len(d_line) for d_line in dis_data[c])
    for line in dis_data[c]:
        if len(line) < maxlen:
            line += [0]*(maxlen-len(line))
    c_data_dict = {str(i): dis_data[c][i] for i in range(len(dis_data[c]))}
    c_data = pd.DataFrame(c_data_dict).T
    c_datas.append(c_data)

In [ ]:
def prepare_pca_data(theme_ques, final_data):
    theme_mean_list = []
    for theme in theme_ques:
        theme_sum = 0
        theme_count = 0
        for ques in theme:
            if ques < len(final_data):
                ques = str(ques)
                if ques not in [7,8] and final_data.loc[ques].sum() != 0:
                    q_mean = 0
                    for choice in final_data.loc[ques].index:
                        if choice != "12":
                            q_mean += float(choice) * (final_data.loc[ques][choice] / final_data.loc[ques].sum())
                    theme_sum += q_mean
                    theme_count+=1
        theme_mean = theme_sum / theme_count
        theme_mean_list.append(theme_mean)
    return theme_mean_list

In [ ]:
us_question_type = pd.read_excel(
    "F:\\Projects\\LLMValueInvestigation\\code\\human_evaluation\\survey_data\\AECE_v1\\Originals\\US_survey\\result_database\\Question_type.xlsx")
type_index = us_question_type.groupby("value type")
us_theme_index = []
for index, value in type_index:
    us_theme_index.append(value["id"].values.tolist())

In [ ]:
import numpy
from sklearn.decomposition import PCA

features = []
for c_data in c_datas:
    features.append(prepare_pca_data(us_theme_index, c_data))

In [ ]:
A = numpy.array(features)
pca = PCA(n_components=2)
pca.fit(A)
result = pca.transform(A)

pca_datapoint = {
    "Survival v.s. Self-Expression": result[:,0],
    "Traditional v.s. Secular": result[:,1],
    "Country": country_list
}
cord = pd.DataFrame(pca_datapoint, index=country_list).drop_duplicates()

In [ ]:
import matplotlib.pyplot as plt

sns.set_context("notebook")
p1 = sns.scatterplot(
    x=cord["Survival v.s. Self-Expression"], 
    y=cord["Traditional v.s. Secular"], 
    data=cord, hue=cord["Country"], 
    palette ='tab20',sizes=[100]
)
for line in range(0,cord.shape[0]):
     p1.text(
         cord["Survival v.s. Self-Expression"][line], 
         cord["Traditional v.s. Secular"][line]-0.01, 
         cord.index[line], 
         horizontalalignment='left', size='small', color='black'
     )
plt.legend(ncols=3, loc="upper left", bbox_to_anchor=(1, 1))
# sns.move_legend(p1, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
import seaborn as sns
# filtered_cord = pd.DataFrame(column=cord.column, index=[])
filter = []
for i in cord.index:
    c = cord.loc[i]
    if c["Survival v.s. Self-Expression"] >= 0 and c["Traditional v.s. Secular"] < 0:
        filter.append(c)
filtered_cord = pd.concat(filter, axis=1)
filtered_data = pd.DataFrame(filtered_cord).T
filtered_data


In [ ]:
sns.set_context("notebook")
p1 = sns.scatterplot(
    x=filtered_data["Survival v.s. Self-Expression"], 
    y=filtered_data["Traditional v.s. Secular"], 
    data=filtered_data, hue=filtered_data["Country"], 
    palette ='tab20',sizes=[100]
)
for line in range(0,filtered_data.shape[0]):
     p1.text(
         filtered_data["Survival v.s. Self-Expression"][line], 
         filtered_data["Traditional v.s. Secular"][line]-0.01, 
         filtered_data.index[line], 
         horizontalalignment='left', size='small', color='black'
     )
sns.move_legend(p1, "upper left", bbox_to_anchor=(1, 1))